## Research Questions

### Economics questions

* How are the employees of the City of Chicago paid by job title relative to other areas with that job in the US?
  * We will need data at the level of the City of Chicago and at a national level, perhaps that supplied by the Bureau of Labor Statistics.
  

### Gender-related questions

If the dataset we start with doesn't have an indication of gender, how will we estimate it?

Could we infer gender from first name?

* How are men with the same job titles in the same departments paid relative to women in the City of Chicago?

* Do women do different jobs than men in the City?

* Are women or men in management roles proportional to the gender distribution of the department?
  * How do we define which roles are management roles?
  
Considerations: 

* How would time spent in the department, position, or career affect compensation?

### Compensation Questions

* Do people with common names get paid differently from people with uncommon names?
  
  
### Safety-related questions

* What are the safest jobs in the City?
  * How do we define 'safe'? Where can we get data?

  
## Informational Views

* Show employees by department.
  * Enrichment idea: Merge in a description of department data so that we have a better idea of where people work.
* Show employees by job title.
* Show the salary of an individual by first or last name.

## We'll start with the "Name, Salary, Title" dataset from the Chicago Data Portal

https://data.cityofchicago.org/Administration-Finance/Current-Employee-Names-Salaries-and-Position-Title/xzkq-xp2w

> This dataset is a listing of all current City of Chicago employees, complete with full names, departments, positions, employment status (part-time or full-time), frequency of hourly employee –where applicable—and annual salaries or hourly rate. For hourly employees, the City is providing the hourly rate and frequency of hourly employees (40, 35, 20 and 10) to allow dataset users to estimate annual wages for hourly employees. Please note that annual wages will vary by employee, depending on number of hours worked and seasonal status. For information on the positions and related salaries detailed in the annual budgets, see https://www.cityofchicago.org/city/en/depts/obm.html


## Data Cleansing - Getting to Tidy Data

We'll need to explore this dataset to see what it offers. We can tell from the description that, at a minimum, we're going to need to do some work "to estimate annual wages for hourly employees". We want to be able to make comparisons of salary between groups for either of our research questions.

In [267]:
import seaborn as sns
sns.set()

import pandas as pd

In [268]:
# Enable the notebook to automatically reload. 
# This makes it possible to move your code to the src folder and to use libraries you
# pip install along the way.

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [269]:
%matplotlib inline

In [270]:
data = pd.read_csv('../data/raw/chicago-salaries.csv')

In [273]:
data.head()

,Name,Job Titles,Department,Full or Part-Time,Salary or Hourly,Typical Hours,Annual Salary,Hourly Rate
0,"AARON, JEFFERY M",SERGEANT,POLICE,F,Salary,NaN,101442.0,NaN
1,"AARON, KARINA",POLICE OFFICER (ASSIGNED AS DETECTIVE),POLICE,F,Salary,NaN,94122.0,NaN
2,"AARON, KIMBERLEI R",CHIEF CONTRACT EXPEDITER,GENERAL SERVICES,F,Salary,NaN,101592.0,NaN
3,"ABAD JR, VICENTE M",CIVIL ENGINEER IV,WATER MGMNT,F,Salary,NaN,110064.0,NaN
4,"ABASCAL, REECE E",TRAFFIC CONTROL AIDE-HOURLY,OEMC,P,Hourly,20.0,NaN,19.86


In [275]:
data.describe(include='all')

,Name,Job Titles,Department,Full or Part-Time,Salary or Hourly,Typical Hours,Annual Salary,Hourly Rate
count,33183,33183,33183,33183,33183,8022.000000,25161.000000,8022.000000
unique,32880,1111,35,2,2,NaN,NaN,NaN
top,"KELLY, MICHAEL J",POLICE OFFICER,POLICE,F,Salary,NaN,NaN,NaN
freq,4,9520,13414,31090,25161,NaN,NaN,NaN
mean,NaN,NaN,NaN,NaN,NaN,34.507604,86786.999790,32.788558
std,NaN,NaN,NaN,NaN,NaN,9.252077,21041.354602,12.112573
min,NaN,NaN,NaN,NaN,NaN,10.000000,7200.000000,2.650000
25%,NaN,NaN,NaN,NaN,NaN,20.000000,76266.000000,21.200000
50%,NaN,NaN,NaN,NaN,NaN,40.000000,90024.000000,35.600000
75%,NaN,NaN,NaN,NaN,NaN,40.000000,96060.000000,40.200000


## Tidy vs. Messy Data

When people talk about "cleaning" data, they really mean structuring the data in such a way as to make initial exploration and analysis easier. Instead of having to constantly re-invent the wheel and tailor your analysis workflow to the shape of every dataset, you reshape the data to fit a standard analysis workflow. Seaborn's plotting functions, for example, are optimized to work with [data in tidy format](https://seaborn.pydata.org/introduction.html#intro-tidy-data).

We want our data set to be "tidy".

Hadley Wickham coined the term [Tidy Data](https://vita.had.co.nz/papers/tidy-data.pdf) to describe data that is clean and ready for analysis. 

* Each variable is a column. A variable contains all values that measure the same underlying attribute (like height, temperature, duration) across units.
* Each observation is a row. An observation contains all values measured on the same unit (like a person, or a day, or a race) across attributes.
* Each type of observational unit is a table 

This is essentially [Codd's 3rd Normal Form](https://en.wikipedia.org/wiki/Third_normal_form) for a single dataset.

Messy data is any other arrangement of the data.

The five most common problems with messy datasets are:

* Column headers are values, not variable names. 
  * [Example](http://python-graph-gallery.com/boxplot/)
  * Solution: [`melt`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.melt.html) the data from wide to long format.
* Multiple variables are stored in one column. This can happen in either the column name or the column values.
  * Examples: 
    * `{'name': 'Shumway, Gordon'}`
    * `[{'sizeChicago': 42, 'sizeNewYork': 8}]`
  * Solution: Split the data so that each variable is a column.
    * `{'first_name': 'Gordon', 'last_name': 'Shumway'}`
    * `[{'size': 42, 'location': 'Chicago'}, {'size': 8, 'location': 'New York'}`
* Variables are stored in both rows and columns.
* Multiple types of observational units are stored in the same table.
  * Solution: Split the data into multiple tables; tidy those tables. Potentially merge them back into a single table.
* A single observational unit is stored in multiple tables.
  * Solution: Merge the data into a single table.


## Data Cleansing TODO List


**Rename columns to allow attribute access in Pandas.**

Pandas provides the ability to access columns directly with [attribute access](http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#attribute-access) indexing. For example, we can write `data.Department`, including auto-complete, instead of `data['Department']`. This is really a matter of convenience for us to work with data in Pandas more easily, but it's an easy win to take unless there's a good reason why we can't rename a column.

This only works if our attributes follow these rules:

* You can use this access only if the index element is a [valid Python identifier](https://docs.python.org/3/reference/lexical_analysis.html#identifiers), e.g. s.1 is not allowed.
* The attribute will not be available if it conflicts with an existing method name, e.g. `s.min` is not allowed.
* Similarly, the attribute will not be available if it conflicts with any of the following list: `index`, `major_axis`, `minor_axis`, `items`.
* In any of these cases, standard indexing will still work, e.g. `s['1']`, `s['min']`, and `s['index']` will access the corresponding element or column.

We can rename each of the columns to a valid Python identifier to make our life easier.


**Split first name, last name, and middle initial.**

Tidy data means the data in each column can't be further split. Doing this with name may not seem to have any immediate benefits; treat it as a cheap speculative bet at this point.


**Handle NaN values for "Full or Part-Time" and "Salary or Hourly" observations.** 

This is an example of the "Multiple types of observational units are stored in the same table" type of mess.

We can see from the `head` preview above that there are `NaN` values. Some of the columns are only defined for Full-Time employees, and some only make sense for Part-Time. Full-time staff typically do different types of work than part-time. Some of part-time work is seasonal. Part-time staff are compensated differently. We'll need to do some work to compare the two types of employees.

There are four possible classes of the dataset:

<table>
    <tr><td>PT Hourly</td><td>PT Salary</td></tr>
    <tr><td>FT Hourly</td><td>FT Salary</td></tr>
</table>

Going back to our definition of tidy data, we want each row to contain all values measured on the same person across attributes. We want to compare groups of rows, not groups of columns, in our analysis. This means each column should have a meaningful value for every row.

The attributes which vary by class are "Typical Hours", "Annual Salary", and "Hourly Rate".

This means we need to ensure that every person has:

* `typical_hours` as 40 for full-time staff
* an `hourly_rate`, which is a function of salary / 52 / 40 for staff paid by salary.
* an `annualized_income`: We can get an approximation for this for hourly staff as a function of Typical Hours, Hourly Rate, and 52 weeks in a year.

**Investigate duplicates.**

We have 33183 entries, but only 32880 unique names. It seems there are ~300 duplicated names. This might be normal...some names are more common than others. We'll want to investigate to see if there's anything we need to do about this.

### Rename columns to allow attribute access in Pandas

In [313]:
# names = {'Name': 'name', 
#          'Job Titles': 'job_title',
#          'Department': ...
#         }

# data = data.rename(index=str, columns=names)

# OR...

# rename can also take a function to run on each of the columns, e.g.
# df.rename(str.lower, axis='columns')

# you could define a function that will replace whitespace and dashes with underscores,
# then pass that function to rename.
import re

def to_snake(s):
    return re.sub("([\s]+|[-]+)", "_", s).lower().lstrip("_")


In [314]:
to_snake("Job Title")

'job_title'

In [315]:
to_snake("Description")

'description'

In [316]:
to_snake("Full or Part-Time")

'full_or_part_time'

In [317]:
data = data.rename(to_snake, axis='columns')

### Split first name, last name, middle initial

In [327]:
no_middle_initial = data.iloc[1]['name']

In [328]:
no_middle_initial

'AARON,  KARINA '

In [374]:
middle_initial = data.iloc[0]['name']

In [331]:
middle_initial

'AARON,  JEFFERY M'

In [334]:
suffix = data.iloc[23]['name']

In [335]:
suffix

'ABERCROMBIE IV,  EARL S'

In [337]:
suffix.split(' ')

['ABERCROMBIE', 'IV,', '', 'EARL', 'S']

In [338]:
middle_initial.split(' ')

['AARON,', '', 'JEFFERY', 'M']

In [339]:
no_middle_initial.split(' ')

['AARON,', '', 'KARINA', '']

In [340]:
suffix.split(',')

['ABERCROMBIE IV', '  EARL S']

In [343]:
no_middle_initial.split(',')

['AARON', '  KARINA ']

In [344]:
names = [no_middle_initial, middle_initial, suffix]

In [342]:
def last_name(s):
    return s.split(',')[0]

In [349]:
data = data.assign(last_name=[last_name(n) for n in data.name])

In [350]:
data.columns

Index(['name', 'job_titles', 'department', 'full_or_part_time',
       'salary_or_hourly', 'typical_hours', 'annual_salary', 'hourly_rate',
       'last_name'],
      dtype='object')

In [352]:
data.head()

,name,job_titles,department,full_or_part_time,salary_or_hourly,typical_hours,annual_salary,hourly_rate,last_name
0,"AARON, JEFFERY M",SERGEANT,POLICE,F,Salary,NaN,101442.0,NaN,AARON
1,"AARON, KARINA",POLICE OFFICER (ASSIGNED AS DETECTIVE),POLICE,F,Salary,NaN,94122.0,NaN,AARON
2,"AARON, KIMBERLEI R",CHIEF CONTRACT EXPEDITER,GENERAL SERVICES,F,Salary,NaN,101592.0,NaN,AARON
3,"ABAD JR, VICENTE M",CIVIL ENGINEER IV,WATER MGMNT,F,Salary,NaN,110064.0,NaN,ABAD JR
4,"ABASCAL, REECE E",TRAFFIC CONTROL AIDE-HOURLY,OEMC,P,Hourly,20.0,NaN,19.86,ABASCAL


In [361]:
temp = middle_initial.split(',')[1].strip()
temp.split(' ')

['JEFFERY', 'M']

In [362]:
temp = no_middle_initial.split(',')[1].strip()
temp.split(' ')

['KARINA']

In [363]:
def first_name(s):
    first_names = s.split(',')[1].strip()
    return first_names.split(' ')[0]

In [364]:
first_name(no_middle_initial)

'KARINA'

In [365]:
first_name(middle_initial)

'JEFFERY'

In [366]:
first_name(suffix)

'EARL'

In [367]:
data = data.assign(first_name=[first_name(n) for n in data.name])

In [368]:
def middle_initial(s):

,name,job_titles,department,full_or_part_time,salary_or_hourly,typical_hours,annual_salary,hourly_rate,last_name,first_name
0,"AARON, JEFFERY M",SERGEANT,POLICE,F,Salary,NaN,101442.0,NaN,AARON,JEFFERY
1,"AARON, KARINA",POLICE OFFICER (ASSIGNED AS DETECTIVE),POLICE,F,Salary,NaN,94122.0,NaN,AARON,KARINA
2,"AARON, KIMBERLEI R",CHIEF CONTRACT EXPEDITER,GENERAL SERVICES,F,Salary,NaN,101592.0,NaN,AARON,KIMBERLEI
3,"ABAD JR, VICENTE M",CIVIL ENGINEER IV,WATER MGMNT,F,Salary,NaN,110064.0,NaN,ABAD JR,VICENTE
4,"ABASCAL, REECE E",TRAFFIC CONTROL AIDE-HOURLY,OEMC,P,Hourly,20.0,NaN,19.86,ABASCAL,REECE


In [375]:
def middle_name(s):
    first_names = s.split(',')[1].strip()
    maybe_first_names = first_names.split(' ')
    if len(maybe_first_names) > 1:
        return maybe_first_names[1]
    return ' '

In [376]:
middle_name(middle_initial)

'M'

In [377]:
middle_name(no_middle_initial)

' '

In [378]:
middle_name(suffix)

'S'

In [379]:
data = data.assign(middle_name=[middle_name(n) for n in data.name])

In [380]:
data.head()

,name,job_titles,department,full_or_part_time,salary_or_hourly,typical_hours,annual_salary,hourly_rate,last_name,first_name,middle_name
0,"AARON, JEFFERY M",SERGEANT,POLICE,F,Salary,NaN,101442.0,NaN,AARON,JEFFERY,M
1,"AARON, KARINA",POLICE OFFICER (ASSIGNED AS DETECTIVE),POLICE,F,Salary,NaN,94122.0,NaN,AARON,KARINA,
2,"AARON, KIMBERLEI R",CHIEF CONTRACT EXPEDITER,GENERAL SERVICES,F,Salary,NaN,101592.0,NaN,AARON,KIMBERLEI,R
3,"ABAD JR, VICENTE M",CIVIL ENGINEER IV,WATER MGMNT,F,Salary,NaN,110064.0,NaN,ABAD JR,VICENTE,M
4,"ABASCAL, REECE E",TRAFFIC CONTROL AIDE-HOURLY,OEMC,P,Hourly,20.0,NaN,19.86,ABASCAL,REECE,E


In [444]:
def hours(row):
    h = row['typical_hours']
    return 40.0 if pd.isna(h) else h


new_hours = pd.DataFrame({'typical_hours': [hours(row) for _, row in data.iterrows()]})

In [445]:
new_hours.describe()

,typical_hours
count,33183.000000
mean,38.672212
std,5.120740
min,10.000000
25%,40.000000
50%,40.000000
75%,40.000000
max,40.000000


In [417]:
data.update(new_hours)

In [418]:
data.head()

,name,job_titles,department,full_or_part_time,salary_or_hourly,typical_hours,annual_salary,hourly_rate,last_name,first_name,middle_name
0,"AARON, JEFFERY M",SERGEANT,POLICE,F,Salary,40.0,101442.0,NaN,AARON,JEFFERY,M
1,"AARON, KARINA",POLICE OFFICER (ASSIGNED AS DETECTIVE),POLICE,F,Salary,40.0,94122.0,NaN,AARON,KARINA,
2,"AARON, KIMBERLEI R",CHIEF CONTRACT EXPEDITER,GENERAL SERVICES,F,Salary,40.0,101592.0,NaN,AARON,KIMBERLEI,R
3,"ABAD JR, VICENTE M",CIVIL ENGINEER IV,WATER MGMNT,F,Salary,40.0,110064.0,NaN,ABAD JR,VICENTE,M
4,"ABASCAL, REECE E",TRAFFIC CONTROL AIDE-HOURLY,OEMC,P,Hourly,20.0,NaN,19.86,ABASCAL,REECE,E


Get every employee that doesn't have an hourly rate.
Do any of them not have an annual salary?

In [427]:
data[data.hourly_rate.isna()].annual_salary.isna().value_counts()

False    25161
Name: annual_salary, dtype: int64

In [430]:
def rate(row):
    r = row['hourly_rate']
    if pd.isna(r):
        return row['annual_salary'] / (52.0 * 40.0)
    return r

In [431]:
new_rate = pd.DataFrame({'hourly_rate': [rate(row) for _, row in data.iterrows()]})

In [432]:
data.update(new_rate)

In [435]:
data.head()

,name,job_titles,department,full_or_part_time,salary_or_hourly,typical_hours,annual_salary,hourly_rate,last_name,first_name,middle_name
0,"AARON, JEFFERY M",SERGEANT,POLICE,F,Salary,40.0,101442.0,48.770192,AARON,JEFFERY,M
1,"AARON, KARINA",POLICE OFFICER (ASSIGNED AS DETECTIVE),POLICE,F,Salary,40.0,94122.0,45.250962,AARON,KARINA,
2,"AARON, KIMBERLEI R",CHIEF CONTRACT EXPEDITER,GENERAL SERVICES,F,Salary,40.0,101592.0,48.842308,AARON,KIMBERLEI,R
3,"ABAD JR, VICENTE M",CIVIL ENGINEER IV,WATER MGMNT,F,Salary,40.0,110064.0,52.915385,ABAD JR,VICENTE,M
4,"ABASCAL, REECE E",TRAFFIC CONTROL AIDE-HOURLY,OEMC,P,Hourly,20.0,NaN,19.860000,ABASCAL,REECE,E


In [436]:
def salary(row):
    r = row['annual_salary']
    if pd.isna(r):
        return row['typical_hours'] * row['hourly_rate'] * 52
    return r

In [437]:
income = [salary(row) for _, row in data.iterrows()]

In [440]:
data = data.assign(annualized_income=income)

In [441]:
data.head()

,name,job_titles,department,full_or_part_time,salary_or_hourly,typical_hours,annual_salary,hourly_rate,last_name,first_name,middle_name,annualized_income
0,"AARON, JEFFERY M",SERGEANT,POLICE,F,Salary,40.0,101442.0,48.770192,AARON,JEFFERY,M,101442.0
1,"AARON, KARINA",POLICE OFFICER (ASSIGNED AS DETECTIVE),POLICE,F,Salary,40.0,94122.0,45.250962,AARON,KARINA,,94122.0
2,"AARON, KIMBERLEI R",CHIEF CONTRACT EXPEDITER,GENERAL SERVICES,F,Salary,40.0,101592.0,48.842308,AARON,KIMBERLEI,R,101592.0
3,"ABAD JR, VICENTE M",CIVIL ENGINEER IV,WATER MGMNT,F,Salary,40.0,110064.0,52.915385,ABAD JR,VICENTE,M,110064.0
4,"ABASCAL, REECE E",TRAFFIC CONTROL AIDE-HOURLY,OEMC,P,Hourly,20.0,NaN,19.860000,ABASCAL,REECE,E,20654.4


In [442]:
data.describe()

,typical_hours,annual_salary,hourly_rate,annualized_income
count,33183.000000,25161.000000,33183.000000,33183.000000
mean,38.672212,86786.999790,39.564248,81069.414879
std,5.120740,21041.354602,11.300300,26023.868229
min,10.000000,7200.000000,2.650000,2756.000000
25%,40.000000,76266.000000,34.860577,72510.000000
50%,40.000000,90024.000000,41.829808,87006.000000
75%,40.000000,96060.000000,46.100000,95888.000000
max,40.000000,300000.000000,144.230769,300000.000000


In [451]:
data[data.duplicated(['name'], keep=False)]

,name,job_titles,department,full_or_part_time,salary_or_hourly,typical_hours,annual_salary,hourly_rate,last_name,first_name,middle_name,annualized_income
173,"ADE, JAMES P",POLICE OFFICER,POLICE,F,Salary,40.0,84054.00,40.410577,ADE,JAMES,P,84054.00
174,"ADE, JAMES P",SERGEANT,POLICE,F,Salary,40.0,111474.00,53.593269,ADE,JAMES,P,111474.00
735,"ANDERSON, DAVID C",PROJECTS ADMINISTRATOR,BUILDINGS,F,Salary,40.0,94824.00,45.588462,ANDERSON,DAVID,C,94824.00
736,"ANDERSON, DAVID C",SEWER BRICKLAYER,WATER MGMNT,F,Hourly,40.0,NaN,44.880000,ANDERSON,DAVID,C,93350.40
742,"ANDERSON, DONALD",FOREMAN OF WATER PIPE CONSTRUCTION,WATER MGMNT,F,Hourly,40.0,NaN,50.250000,ANDERSON,DONALD,,104520.00
743,"ANDERSON, DONALD",PROJECT MANAGER,AVIATION,F,Salary,40.0,80868.00,38.878846,ANDERSON,DONALD,,80868.00
788,"ANDERSON, RHONDA M",PROPERTY CUSTODIAN - AUTO POUND,STREETS & SAN,F,Salary,40.0,43644.00,20.982692,ANDERSON,RHONDA,M,43644.00
789,"ANDERSON, RHONDA M",SERGEANT,POLICE,F,Salary,40.0,104628.00,50.301923,ANDERSON,RHONDA,M,104628.00
1051,"ARROYO, FRANCISCO",LIBRARIAN II,PUBLIC LIBRARY,F,Salary,40.0,84516.00,40.632692,ARROYO,FRANCISCO,,84516.00
1052,"ARROYO, FRANCISCO",WARD SUPERINTENDENT,STREETS & SAN,F,Salary,40.0,76716.00,36.882692,ARROYO,FRANCISCO,,76716.00


### Data Enrichment: Gender

We'll use the [gender-guesser](https://github.com/lead-ratings/gender-guesser) package to match our first names to a gender. According to the documentation, 

> "The result will be one of `unknown` (name not found), `andy` (androgynous), `male`, `female`, `mostly_male`, or `mostly_female`. The difference between `andy` and `unknown` is that the former is found to have the same probability to be male than to be female, while the later means that the name wasn't found in the database.

Let's try it out.

In [502]:
import gender_guesser.detector as gender

detector = gender.Detector(case_sensitive=False)

In [503]:
detector.get_gender('Bobby')

'male'

In [504]:
detector.get_gender('Sarah')

'female'

In [505]:
detector.get_gender('Pat')

'andy'

In [509]:
detector.get_gender('Aidin')

'unknown'

In [510]:
detector.get_gender('Kelly')

'mostly_female'

In [514]:
detector.get_gender('Gene')

'mostly_male'

Let's see how this package performs with our list of first names: